In [1]:
import osmapi as osm
import numpy as np
import pandas as pd
import plotly as pt
import osmapi as osm
import plotly.graph_objects as go
import plotly.express as px
import pydeck as pdk
import folium
import os
import datetime
api = osm.OsmApi()

In [2]:
data = pd.read_csv("track.csv")
data


,Unnamed: 0,id,time,geometry,Speed.value,Speed.unit,Throttle Position.value,Throttle Position.unit,GPS VDOP.value,GPS VDOP.unit,...,track.appVersion,track.touVersion,O2 Lambda Voltage ER.value,O2 Lambda Voltage ER.unit,MAF.value,MAF.unit,O2 Lambda Voltage.value,O2 Lambda Voltage.unit,st_nodes,end_nodes
0,0,5eb7582165b80c5d6be6a144,2020-05-09T08:12:25,POINT (7.653064001351595 51.97227354627103),0.000000,km/h,18.000000,%,0.961303,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129974501,6237445823
1,1,5eb7582165b80c5d6be6a146,2020-05-09T08:12:30,POINT (7.653108948898172 51.97234716083879),9.000000,km/h,16.000000,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129974501,6237445823
2,2,5eb7582165b80c5d6be6a147,2020-05-09T08:12:35,POINT (7.652941338383542 51.97242707033375),15.769492,km/h,17.000000,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6237445823,129974359
3,3,5eb7582165b80c5d6be6a148,2020-05-09T08:12:40,POINT (7.65249814269891 51.97243434596708),20.054118,km/h,17.709184,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129974359,129974355
4,4,5eb7582165b80c5d6be6a149,2020-05-09T08:12:45,POINT (7.652107879570824 51.97230765126906),21.726776,km/h,18.341146,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129974355,129974330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,105,5eb7582165b80c5d6be6a1ae,2020-05-09T08:21:15,POINT (7.651989999238601 51.95400700811021),10.000000,km/h,15.000000,%,0.912846,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244517237,262571338
106,106,5eb7582165b80c5d6be6a1af,2020-05-09T08:21:20,POINT (7.651829345262824 51.95399070884194),4.711911,km/h,14.000000,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244517237,262571338
107,107,5eb7582165b80c5d6be6a1b0,2020-05-09T08:21:25,POINT (7.651801770552993 51.95398047687347),0.000000,km/h,14.000000,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244517237,262571338
108,108,5eb7582165b80c5d6be6a1b1,2020-05-09T08:21:30,POINT (7.651801770552993 51.9539804616943),0.000000,km/h,14.000000,%,0.800000,precision,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244517237,262571338


In [3]:
data.dtypes

Unnamed: 0                      int64
id                             object
time                           object
geometry                       object
Speed.value                   float64
Speed.unit                     object
Throttle Position.value       float64
Throttle Position.unit         object
GPS VDOP.value                float64
GPS VDOP.unit                  object
GPS Altitude.value            float64
GPS Altitude.unit              object
CO2.value                     float64
CO2.unit                       object
Intake Pressure.value         float64
Intake Pressure.unit           object
Calculated MAF.value          float64
Calculated MAF.unit            object
Intake Temperature.value      float64
Intake Temperature.unit        object
GPS PDOP.value                float64
GPS PDOP.unit                  object
GPS Speed.value               float64
GPS Speed.unit                 object
GPS Bearing.value             float64
GPS Bearing.unit               object
GPS Accuracy

In [4]:
edgeset = pd.DataFrame({'st_node': data["st_nodes"], 'end_node': data["end_nodes"], 'speed':data["Speed.value"], 'time':data["time"]})
edgeset

,st_node,end_node,speed,time
0,129974501,6237445823,0.000000,2020-05-09T08:12:25
1,129974501,6237445823,9.000000,2020-05-09T08:12:30
2,6237445823,129974359,15.769492,2020-05-09T08:12:35
3,129974359,129974355,20.054118,2020-05-09T08:12:40
4,129974355,129974330,21.726776,2020-05-09T08:12:45
...,...,...,...,...
105,244517237,262571338,10.000000,2020-05-09T08:21:15
106,244517237,262571338,4.711911,2020-05-09T08:21:20
107,244517237,262571338,0.000000,2020-05-09T08:21:25
108,244517237,262571338,0.000000,2020-05-09T08:21:30


In [5]:
def aggregateStatsFromNodes(edgeset):
    edge_agg = edgeset.groupby(['st_node', 'end_node']).agg(mean_speed=pd.NamedAgg(column='speed', aggfunc=np.mean),
                                                           min_speed=pd.NamedAgg(column='speed', aggfunc=min),
                                                           max_speed=pd.NamedAgg(column='speed', aggfunc=max))
    edgeset = pd.merge(edgeset, edge_agg,  how='left', left_on=['st_node','end_node'], right_on = ['st_node','end_node'])
    edgeset = edgeset.drop(['speed'], axis=1).drop_duplicates()
    return edgeset
    
edgeset = aggregateStatsFromNodes(edgeset)

In [6]:
def appendNodeCoords(edgeset):
    arrStartNodelat = np.array([])
    arrStartNodelon = np.array([])
    arrEndNodelat = np.array([])
    arrEndNodelon = np.array([])

    
    for row in edgeset.itertuples():
        
        stNode = api.NodeGet(row.st_node)
        arrStartNodelat = np.append(arrStartNodelat, stNode["lat"])
        arrStartNodelon = np.append(arrStartNodelon, stNode["lon"])
    
        endNode = api.NodeGet(row.end_node)
        arrEndNodelat = np.append(arrEndNodelat,endNode["lat"])
        arrEndNodelon = np.append(arrEndNodelon, endNode["lon"])
        
        
    
    edgeset['StNode_lat']=arrStartNodelat
    edgeset['StNode_lon']=arrStartNodelon
    edgeset['EndNode_lat']=arrEndNodelat
    edgeset['EndNode_lon']=arrEndNodelon
    
    
appendNodeCoords(edgeset)
edgeset    

,st_node,end_node,time,mean_speed,min_speed,max_speed,StNode_lat,StNode_lon,EndNode_lat,EndNode_lon
0,129974501,6237445823,2020-05-09T08:12:25,4.500000,0.000000,9.000000,51.971903,7.652766,51.972420,7.653054
1,129974501,6237445823,2020-05-09T08:12:30,4.500000,0.000000,9.000000,51.971903,7.652766,51.972420,7.653054
2,6237445823,129974359,2020-05-09T08:12:35,15.769492,15.769492,15.769492,51.972420,7.653054,51.972422,7.652957
3,129974359,129974355,2020-05-09T08:12:40,20.054118,20.054118,20.054118,51.972422,7.652957,51.972422,7.652484
4,129974355,129974330,2020-05-09T08:12:45,25.908925,21.726776,29.000000,51.972422,7.652484,51.971733,7.651412
...,...,...,...,...,...,...,...,...,...,...
105,244517237,262571338,2020-05-09T08:21:15,6.451985,0.000000,12.000000,51.953944,7.652730,51.953929,7.650828
106,244517237,262571338,2020-05-09T08:21:20,6.451985,0.000000,12.000000,51.953944,7.652730,51.953929,7.650828
107,244517237,262571338,2020-05-09T08:21:25,6.451985,0.000000,12.000000,51.953944,7.652730,51.953929,7.650828
108,244517237,262571338,2020-05-09T08:21:30,6.451985,0.000000,12.000000,51.953944,7.652730,51.953929,7.650828


In [7]:
def plotAggregatedStatistics(edgeset, speed):
    INITIAL_VIEW_STATE = pdk.ViewState(latitude=51.96, longitude=7.64, zoom=15, max_zoom=20, pitch=50, bearing=0)

    
    if speed:
        pointcolor='[mean_speed < 10 ? 50 : (mean_speed < 20 ? 100 :(mean_speed < 30 ? 150:200)),0,0,255]' 
        linecolor='[0,mean_speed < 10 ? 50 : (mean_speed < 20 ? 100 :(mean_speed < 30 ? 150:200)),0,255]'

    scatterplot = pdk.Layer(
        "ScatterplotLayer",
        edgeset,
        radius_scale=10,
        get_position=['StNode_lon', 'StNode_lat'],
        get_fill_color=pointcolor,
        get_radius=1,
        pickable=True,
    )

  
    
    line_layer = pdk.Layer(
        "LineLayer",
        edgeset,
        get_source_position=['StNode_lon', 'StNode_lat'],
        get_target_position=['EndNode_lon', 'EndNode_lat'],
        get_color= linecolor,
        get_width=10,
        highlight_color=[255, 255, 0],
        picking_radius=10,
        auto_highlight=True,
        pickable=True,
    )

    layers = [line_layer,scatterplot]
    MAPBOX_KEY = "pk.eyJ1IjoibXByZW1hc2kiLCJhIjoiY2s5NDFueDhyMDFpODNnbjNoNzM1eWhvcCJ9.CqjZdNZJ4h8aejMWX4ZObA"
    r = pdk.Deck(layers=layers, initial_view_state=INITIAL_VIEW_STATE,mapbox_key = MAPBOX_KEY)
    r.to_html("line_layer.html", iframe_width=900)
    
plotAggregatedStatistics(edgeset,True)

Handling Multiple Tracks

Use Case - Mean speed of streets on Saturday between 08:15 to 08:18 
- Concatenate Map Matched Tracks
- Remove unwanted fields
- Convert time field to time object field
- Filter tracks by date and Time
- Aggregates Nodes
- Mapping

In [8]:
data1 = pd.read_csv("track.csv")
data2 = pd.read_csv("track.csv")

frames = [data1, data2]
frames

concatTracks = pd.concat(frames)
concatTracks = pd.DataFrame({'st_node': concatTracks["st_nodes"], 'end_node': concatTracks["end_nodes"], 'speed':concatTracks["Speed.value"],'time':concatTracks["time"]})
pd.set_option("display.max_rows", None, "display.max_columns", None)
concatTracks





,st_node,end_node,speed,time
0,129974501,6237445823,0.000000,2020-05-09T08:12:25
1,129974501,6237445823,9.000000,2020-05-09T08:12:30
2,6237445823,129974359,15.769492,2020-05-09T08:12:35
3,129974359,129974355,20.054118,2020-05-09T08:12:40
4,129974355,129974330,21.726776,2020-05-09T08:12:45
5,129974355,129974330,27.000000,2020-05-09T08:12:50
6,129974355,129974330,29.000000,2020-05-09T08:12:55
7,129974330,129974324,27.078432,2020-05-09T08:13:00
8,129974324,129974313,22.761039,2020-05-09T08:13:06
9,129974324,129974313,11.086487,2020-05-09T08:13:11


In [12]:
concatTracks['time'] = pd.to_datetime(concatTracks['time'])
concatTracks.dtypes

concatTracks['weekday'] = concatTracks['time'].dt.dayofweek 
#Filter Saturday
maskDay = (concatTracks['weekday'] == 5)
filteredConcatTracksDay = concatTracks.loc[maskDay]
filteredConcatTracksDay

if not filteredConcatTracksDay.empty:
    maskTime = (filteredConcatTracksDay['time'] > '2019-05-08 08:15:00') & (filteredConcatTracksDay['time'] <= '2020-05-09 08:18:00')
    filteredConcatTracksDayTime = filteredConcatTracksDay.loc[maskTime]
    print(filteredConcatTracksDayTime)
   

       st_node    end_node      speed                time  weekday
0    129974501  6237445823   0.000000 2020-05-09 08:12:25        5
1    129974501  6237445823   9.000000 2020-05-09 08:12:30        5
2   6237445823   129974359  15.769492 2020-05-09 08:12:35        5
3    129974359   129974355  20.054118 2020-05-09 08:12:40        5
4    129974355   129974330  21.726776 2020-05-09 08:12:45        5
5    129974355   129974330  27.000000 2020-05-09 08:12:50        5
6    129974355   129974330  29.000000 2020-05-09 08:12:55        5
7    129974330   129974324  27.078432 2020-05-09 08:13:00        5
8    129974324   129974313  22.761039 2020-05-09 08:13:06        5
9    129974324   129974313  11.086487 2020-05-09 08:13:11        5
10   129974324   129974313   7.000000 2020-05-09 08:13:16        5
11   129974324   129974313   8.022500 2020-05-09 08:13:21        5
12   129974313   115790055  13.894865 2020-05-09 08:13:26        5
13   115790055    60778426  30.760919 2020-05-09 08:13:31     

In [10]:
concatAggregatedTracks = aggregateStatsFromNodes(filteredConcatTracksDayTime)
appendNodeCoords(concatAggregatedTracks)
concatAggregatedTracks


,st_node,end_node,time,weekday,mean_speed,min_speed,max_speed,StNode_lat,StNode_lon,EndNode_lat,EndNode_lon
0,129974501,6237445823,2020-05-09 08:12:25,5,4.500000,0.000000,9.000000,51.971903,7.652766,51.972420,7.653054
1,129974501,6237445823,2020-05-09 08:12:30,5,4.500000,0.000000,9.000000,51.971903,7.652766,51.972420,7.653054
2,6237445823,129974359,2020-05-09 08:12:35,5,15.769492,15.769492,15.769492,51.972420,7.653054,51.972422,7.652957
3,129974359,129974355,2020-05-09 08:12:40,5,20.054118,20.054118,20.054118,51.972422,7.652957,51.972422,7.652484
4,129974355,129974330,2020-05-09 08:12:45,5,25.908925,21.726776,29.000000,51.972422,7.652484,51.971733,7.651412
5,129974355,129974330,2020-05-09 08:12:50,5,25.908925,21.726776,29.000000,51.972422,7.652484,51.971733,7.651412
6,129974355,129974330,2020-05-09 08:12:55,5,25.908925,21.726776,29.000000,51.972422,7.652484,51.971733,7.651412
7,129974330,129974324,2020-05-09 08:13:00,5,27.078432,27.078432,27.078432,51.971733,7.651412,51.971334,7.651152
8,129974324,129974313,2020-05-09 08:13:06,5,12.217506,7.000000,22.761039,51.971334,7.651152,51.970492,7.650731
9,129974324,129974313,2020-05-09 08:13:11,5,12.217506,7.000000,22.761039,51.971334,7.651152,51.970492,7.650731


In [11]:

plotAggregatedStatistics(concatAggregatedTracks,True)